In [ ]:
import numpy as np
import pandas as pd
from yaml import safe_load
import os
from tqdm import tqdm

In [ ]:
filenames = []
for file in os.listdir('/content/drive/MyDrive/cricket score predictions '):
    filenames.append(os.path.join('/content/drive/MyDrive/cricket score predictions ',file))

In [ ]:
# %%capture
final_df = pd.DataFrame()
counter = 1
for file in tqdm(filenames):
    with open(file, 'r') as f:
        df = pd.json_normalize(safe_load(f))
        df['match_id'] = counter
        final_df = final_df.append(df)
        counter+=1

In [ ]:
backup = final_df.copy()

In [ ]:
final_df.drop(columns=[
    'meta.data_version',
    'meta.created',
    'meta.revision',
    'info.outcome.bowl_out',
    'info.bowl_out',
    'info.supersubs.South Africa',
    'info.supersubs.New Zealand',
    'info.outcome.eliminator',
    'info.outcome.result',
    'info.outcome.method',
    'info.neutral_venue',
    'info.match_type_number',
    'info.outcome.by.runs',
    'info.outcome.by.wickets'
],inplace=True)

In [ ]:
final_df = final_df[final_df['info.gender'] == 'male']
final_df.drop(columns=['info.gender'],inplace=True)

In [ ]:
final_df = final_df[final_df['info.overs'] == 20]
final_df.drop(columns=['info.overs','info.match_type'],inplace=True)

<ipython-input-35-7fb4794c9448>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.drop(columns=['info.overs','info.match_type'],inplace=True)


In [ ]:
import pickle
pickle.dump(final_df,open('dataset_level1.pkl','wb'))

In [ ]:
matches = pickle.load(open('/content/dataset_level1.pkl','rb'))
matches.iloc[0]['innings'][0]['1st innings']['deliveries']

In [ ]:
count = 1
delivery_df = pd.DataFrame()
for index, row in matches.iterrows():
    if count in [75,108,150,180,268,360,443,458,584,748,982,1052,1111,1226,1345]:
        count+=1
        continue
    count+=1
    ball_of_match = []
    batsman = []
    bowler = []
    runs = []
    player_of_dismissed = []
    teams = []
    batting_team = []
    match_id = []
    city = []
    venue = []
    for ball in row['innings'][0]['1st innings']['deliveries']:
        for key in ball.keys():
            match_id.append(count)
            batting_team.append(row['innings'][0]['1st innings']['team'])
            teams.append(row['info.teams'])
            ball_of_match.append(key)
            batsman.append(ball[key]['batsman'])
            bowler.append(ball[key]['bowler'])
            runs.append(ball[key]['runs']['total'])
            city.append(row['info.city'])
            venue.append(row['info.venue'])
            try:
                player_of_dismissed.append(ball[key]['wicket']['player_out'])
            except:
                player_of_dismissed.append('0')
    loop_df = pd.DataFrame({
            'match_id':match_id,
            'teams':teams,
            'batting_team':batting_team,
            'ball':ball_of_match,
            'batsman':batsman,
            'bowler':bowler,
            'runs':runs,
            'player_dismissed':player_of_dismissed,
            'city':city,
            'venue':venue
        })
    delivery_df = delivery_df.append(loop_df)

In [40]:
def bowl(row):
    for team in row['teams']:
        if team != row['batting_team']:
            return team

In [41]:
delivery_df['bowling_team'] = delivery_df.apply(bowl,axis=1)

In [42]:
delivery_df.drop(columns=['teams'],inplace=True)

In [43]:
delivery_df['batting_team'].unique()

array(['Kenya', 'Botswana', 'Pakistan', 'Denmark', 'Jersey', 'Namibia',
       'West Indies', 'Thailand', 'New Zealand', 'India', 'Norway',
       'South Africa', 'Australia', 'England', 'Maldives', 'Bangladesh',
       'Malaysia', 'Netherlands', 'Guernsey', 'Sri Lanka', 'Kuwait',
       'Nepal', 'Singapore', 'Afghanistan', 'Zimbabwe',
       'United States of America', 'Cayman Islands', 'Canada', 'Oman',
       'Bermuda', 'Hong Kong', 'Ireland', 'Scotland',
       'United Arab Emirates', 'Papua New Guinea', 'Nigeria', 'Portugal',
       'Gibraltar', 'Vanuatu', 'Spain', 'Germany', 'Qatar', 'Iran',
       'Belgium', 'Isle of Man', 'Bulgaria', 'Romania', 'Philippines',
       'Italy', 'Ghana', 'Uganda'], dtype=object)

In [44]:
teams = [
    'Australia',
    'India',
    'Bangladesh',
    'New Zealand',
    'South Africa',
    'England',
    'West Indies',
    'Afghanistan',
    'Pakistan',
    'Sri Lanka'
]

In [45]:
delivery_df = delivery_df[delivery_df['batting_team'].isin(teams)]
delivery_df = delivery_df[delivery_df['bowling_team'].isin(teams)]

In [46]:
output = delivery_df[['match_id','batting_team','bowling_team','ball','runs','player_dismissed','city','venue']]


In [47]:
pickle.dump(output,open('dataset_level2.pkl','wb'))


In [ ]:
output[output['city'].isnull()]['venue'].value_counts()


In [49]:
x = np.where(output['city'].isnull(), output['venue'].str.split().apply(lambda x:x[0]), output['city'])


In [ ]:
output['city'] = x


In [ ]:
output.drop(columns=['venue'],inplace=True)


In [52]:
total_df = output.groupby('match_id').sum()['runs'].reset_index()
output = output.merge(total_df,on='match_id')

<ipython-input-52-7a22a32e7524>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_df = output.groupby('match_id').sum()['runs'].reset_index()


In [53]:
output['current_score'] = output.groupby('match_id').cumsum()['runs_x']


<ipython-input-53-5a28c7ec6464>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.cumsum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  output['current_score'] = output.groupby('match_id').cumsum()['runs_x']


In [54]:
output['over'] = output['ball'].apply(lambda x:str(x).split(".")[0])
output['ball_no'] = output['ball'].apply(lambda x:str(x).split(".")[1])

In [55]:
output['balls_bowled'] = (output['over'].astype('int')*6) + output['ball_no'].astype('int')


In [56]:
output['crr'] = round((output['current_score']*6)/output['balls_bowled'],2)


In [57]:
output['player_dismissed'] = output['player_dismissed'].apply(lambda x:0 if x=='0' else 1)
output['player_dismissed'] = output['player_dismissed'].astype('int')
output['player_dismissed'] = output.groupby('match_id').cumsum()['player_dismissed']
output['wickets_left'] = 10 - output['player_dismissed']

<ipython-input-57-bf2575bb1e2d>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.cumsum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  output['player_dismissed'] = output.groupby('match_id').cumsum()['player_dismissed']


In [58]:
final_df = output[['match_id','batting_team','bowling_team','runs_x','current_score','balls_bowled','wickets_left','crr','city','runs_y']]


In [59]:
final_df = final_df.sample(final_df.shape[0])


In [60]:
final_df['balls_left'] = 120 - final_df['balls_bowled']
final_df['balls_left'] = final_df['balls_left'].apply(lambda x:0 if x<0 else x)

In [61]:
final_df['crr'] = round((final_df['current_score']*6)/final_df['balls_bowled'],2)


In [63]:
final_df.drop(columns=['balls_bowled'],inplace=True)


In [64]:
groups = final_df.groupby('match_id')


In [ ]:
match_ids = final_df['match_id'].unique()
last_five = []
for id in match_ids:
    last_five.extend(groups.get_group(id).rolling(window=30).sum()['runs_x'].values.tolist())

In [66]:
final_df['last_five'] = last_five


In [67]:
final_df.dropna(inplace=True)


In [68]:
eligible_cities = final_df['city'].value_counts()[final_df['city'].value_counts() > 600].index.tolist()
final_df = final_df[final_df['city'].isin(eligible_cities)]

In [69]:
X = final_df.drop(columns=['match_id','runs_x','runs_y'])
y = final_df['runs_y']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [70]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error

trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),['batting_team','bowling_team','city'])
]
,remainder='passthrough')

In [71]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=12,random_state=1))
])

In [72]:
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0.9530144816081954
3.2691531736999346
